**Vamos a ver el df_user_reviews para crear la nueva tabla de análisis de sentimiento**


In [7]:
import pandas as pd 
import numpy as np
import pyarrow as pa

import re
#NLTK es una librería para poder usar NLP
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#Analizador de texto
from textblob import TextBlob


In [8]:
df_user_reviews= pd.read_parquet('df_user_reviews.parquet')

df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58400 entries, 0 to 25798
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       58400 non-null  object
 1   item_id       58400 non-null  object
 2   recommend     58400 non-null  bool  
 3   review        58400 non-null  object
 4   anio_lanzado  48471 non-null  Int64 
dtypes: Int64(1), bool(1), object(3)
memory usage: 2.3+ MB


**Ahora vamos a crear el análisis de sentimientos**

Vader viene de “Valence Aware Dictionary and sEntiment Reasoner” y es la librería que usa Python para el análisis de sentimientos. Esta librería se basa en reglas y un poderoso lexicón para realizar sus análisis y determinar si una noticia es positiva, negativa y neutra.

In [13]:
nltk.download('vader_lexicon')

#Se inicializa el analizador

sia = SentimentIntensityAnalyzer()

#Función para designar valores
def sentimiento(txt):
    if pd.isnull(txt) or txt == '':
        return 1 #Se designa que es 1 si está vació o es nulo
    elif isinstance(txt,str):
        #Aqui utiizamos Vader
        vader = sia.polarity_scores(txt)
        vader_puntos=vader['compound']
        
        #Se analiza el texto utilizando TextBlob
        textblob = TextBlob(txt)
        polaridad=textblob.sentiment.polarity
        
        #Escalas para asignar valores
        
        if vader_puntos >= -0.05 and polaridad >= -0.2 and polaridad <=0.2:
            return 1 #Neutral
        elif vader_puntos >= -0.05 or polaridad >0.2:
            return 2 #Positivo
        elif vader_puntos <= -0.05 or polaridad < -.2:
            return 0 #Negatuvo
        
    return 1 

#Aplicamos la función mientras creamos la nueva columna

df_user_reviews['Sentiment_Score'] = df_user_reviews['review'].apply(sentimiento)
   
   
print(df_user_reviews)   

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                 user_id item_id  recommend  \
0      76561197970982479    1250       True   
0      76561197970982479   22200       True   
0      76561197970982479   43110       True   
1                js41637  251610       True   
1                js41637  227300       True   
...                  ...     ...        ...   
25797  76561198312638244      70       True   
25797  76561198312638244  362890       True   
25798        LydiaMorley  273110       True   
25798        LydiaMorley     730       True   
25798        LydiaMorley     440       True   

                                                  review  anio_lanzado  \
0      Simple yet with great replayability. In my opi...          2011   
0                   It's unique and worth a playthrough.          2011   
0      Great atmosphere. The gunplay can be a bit chu...          2011   
1      I know what you think when you see this title ...          2014   
1      For a simple (it's actually not all that simpl...         

Vamos a ver los idiomas que tienen las columnas

Hay que instalar la librería de langdetect para que esta reconozca los caracteres que tiene cada lengua en particular

In [ ]:
pip install langdetect


In [19]:
from langdetect import detect

In [20]:
def idioma (txt):
    try:
        return detect(txt)
    except:
        return "No se encuentra"
    
df_user_reviews['idioma'] = df_user_reviews ['review'].apply(idioma) 

df_user_reviews 

,user_id,item_id,recommend,review,anio_lanzado,Sentiment_Score,idioma
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,1,en
0,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2,en
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1,en
1,js41637,251610,True,I know what you think when you see this title ...,2014,1,en
1,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,1,en
...,...,...,...,...,...,...,...
25797,76561198312638244,70,True,a must have classic from steam definitely wort...,<NA>,2,en
25797,76561198312638244,362890,True,this game is a perfect remake of the original ...,<NA>,1,en
25798,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,<NA>,1,en
25798,LydiaMorley,730,True,:D,<NA>,2,de


Vamos a ver que idiomas hay en nuestros analisis y si vamos a limpiarlas o no

In [21]:
variedad = df_user_reviews['idioma'].value_counts()

print(variedad)

idioma
en                 45073
pt                  2155
es                  1259
de                  1151
so                  1003
af                   737
th                   708
tl                   575
No se encuentra      543
cy                   450
da                   399
no                   365
nl                   307
ca                   261
it                   247
ro                   244
pl                   238
id                   238
ko                   233
fr                   222
zh-cn                210
ru                   210
et                   185
sw                   182
tr                   160
hr                   141
sl                   135
vi                   122
sq                   104
fi                    97
hu                    97
sv                    73
sk                    69
cs                    42
lt                    38
ja                    37
lv                    37
bg                    21
uk                    14
zh-tw             

Ahora vamos a hacer una especie de limpieza de aquellas reseñas que tienen emojis o estrellas. Lo que vamos a hacer es una función Regex para evitar estos problemas

In [22]:
def parsear (txt):
    #Expresiones regex que nos van a decir si es positivo
    positivo =[r"< ?3{3,}", r"♥{3,}", r":\){3,}", r"★{3,}"]
    #Puntaje
    calificacion=[r"\d{3,5}/10", r"[8-9]/10|[1-9][0-9]{1,3}", r"[8-9]/[10-1000]"]
    
    for exp in positivo + calificacion:
        if re.search(exp,txt):
            return 2 # Sentimiento alto
        
    return 1 # Si no se encuentra nada se le asigna el 1   


#creación de mascara para las filas "No se encuentra"
mascara = df_user_reviews['idioma'] == 'No se encuentra'

#Se aplica la función

df_user_reviews.loc[mascara,'Sentiment_Score'] = df_user_reviews.loc[mascara,'review'].apply(parsear)

print(df_user_reviews)

                 user_id item_id  recommend  \
0      76561197970982479    1250       True   
0      76561197970982479   22200       True   
0      76561197970982479   43110       True   
1                js41637  251610       True   
1                js41637  227300       True   
...                  ...     ...        ...   
25797  76561198312638244      70       True   
25797  76561198312638244  362890       True   
25798        LydiaMorley  273110       True   
25798        LydiaMorley     730       True   
25798        LydiaMorley     440       True   

                                                  review  anio_lanzado  \
0      Simple yet with great replayability. In my opi...          2011   
0                   It's unique and worth a playthrough.          2011   
0      Great atmosphere. The gunplay can be a bit chu...          2011   
1      I know what you think when you see this title ...          2014   
1      For a simple (it's actually not all that simpl...         

Eliminamos la columna 'idioma' ya que cumplio su función

In [24]:
df_user_reviews.drop(columns='idioma',inplace=True)

Ahora esto lo exportamos

In [26]:
df_user_reviews_clean = df_user_reviews.to_parquet('df_user_reviews_clean.parquet')

Al final me di cuenta que todo este proceso lo podría haber hecho en ETLFinal para que sea más fácil y no tener muchos archivos pero es parte del aprendizaje. 